# AbsPL (Absorption and Photoluminescence) Analysis

## How to use this notebook:
1. Select batches to analyze (only batches of type "hysprint_batch" are considered)
2. The data will be loaded into a pandas DataFrame
3. Use the plotting tools to visualize your data:
   - Create scatter plots for comparing two parameters
   - Use box plots to analyze parameter distributions
4. Access advanced features for data table viewing and statistics

In [2]:
%matplotlib ipympl
%load_ext autoreload
%autoreload 2
import os
import base64
import io
import sys
import ipywidgets as widgets
import plotly.graph_objects as go
import plotly.express as px
from IPython.display import display, Markdown, HTML
import pandas as pd
import numpy as np
import json

sys.path.append(os.path.dirname(os.getcwd()))
from api_calls import get_ids_in_batch, get_sample_description, get_all_eqe as get_all_abspl
import batch_selection
import plotting_utils
import access_token

url_base ="https://nomad-hzb-se.de"
url = f"{url_base}/nomad-oasis/api/v1"
token = access_token.get_token(url)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload



KeyboardInterrupt



In [ ]:
warning_sign = "\u26A0"

out = widgets.Output()
out2 = widgets.Output()
read = widgets.Output()
dynamic_content = widgets.Output()  # For dynamically updated content
results_content = widgets.Output(layout={
    # 'border': '1px solid black',  # Optional: adds a border to the widget
    'max_height': '1000px',  # Set the height
    'overflow': 'scroll',  # Adds a scrollbar if content overflows
    })
cell_edit = widgets.VBox() 

default_variables = widgets.Dropdown(
    options=['sample name', 'batch',"sample description", 'custom'],
    index=0,
    description='name preset:',
    disabled=False,
    tooltip="Presets for how the samples will be named in the plot"
)
data = None
original_data = None  # To store original data for filter reset


#this function takes sample ids and returns the eqe curves and parameters as Dataframes
def get_abspl_data(try_sample_ids, variation):
    #parameters of single eqe measurement
    abs_pl_params_names = ["luminescence_quantum_yield","quasi_fermi_level_splitting","i_voc","bandgap","derived_jsc", "wavelength", "luminescence_flux_density"]
    #make api call, result has everything in json format
    all_abspl = get_all_abspl(url, token, try_sample_ids, eqe_type="HySprint_AbsPLMeasurement")

    existing_sample_ids = pd.Series(all_abspl.keys())

    # Check if there's any EQE data
    if len(existing_sample_ids) == 0:
        return None  # Return None value to indicate no data

    sample_params_list = []
    for sample_id, sample_data in all_abspl.items():
        for abspl_entry in sample_data:
            df = pd.DataFrame(abspl_entry[0]["results"], columns=abs_pl_params_names)
            df["sample_id"] = sample_id
            df["variation"] = variation.get(sample_id, '')
            df["name"] = abspl_entry[0].get("name", '')
            sample_params_list.append(df)
    print(sample_params_list)

      
    # Only try to concatenate if there's data
    if sample_params_list:
        return pd.concat(sample_params_list)
    return None

def create_data_filters():
    """Create interactive filters for numerical values in the DataFrame with reset functionality"""
    global data, original_data
    
    if data is None:
        return widgets.HTML("<h3>Please load data first</h3>")
    
    # Output for filtered data and filter status
    filter_output = widgets.Output()
    filter_status = widgets.Output()
    
    # Get numeric columns for filtering
    numeric_cols = data.select_dtypes(include=['float64', 'int64']).columns.tolist()
    
    # Create column selection dropdown
    column_selector = widgets.Dropdown(
        options=numeric_cols,
        value=numeric_cols[0] if numeric_cols else None,
        description='Filter column:',
        disabled=False
    )
    
    # Create min and max value text inputs instead of sliders
    min_value = widgets.FloatText(
        description='Min value:',
        disabled=False,
        layout=widgets.Layout(width='300px'),
        style={'description_width': '100px'}
    )
    
    max_value = widgets.FloatText(
        description='Max value:',
        disabled=False,
        layout=widgets.Layout(width='300px'),
        style={'description_width': '100px'}
    )
    
    # Create apply filter button
    apply_button = widgets.Button(
        description='Apply Filter',
        button_style='success',
        tooltip='Click to apply the filter'
    )
    
    # Create reset button
    reset_button = widgets.Button(
        description='Reset All Filters',
        button_style='danger',
        tooltip='Click to reset all filters and restore original data'
    )
    
    # Add button to show current data
    show_data_button = widgets.Button(
        description='Show Filtered Data',
        button_style='info',
        tooltip='Click to view current filtered data'
    )
    
    # Function to update min/max input values when column changes
    def update_input_values(change):
        if change.new and change.new in data.columns:
            col = change.new
            min_val = data[col].min()
            max_val = data[col].max()
            
            # Update the text input fields with min and max values
            min_value.value = min_val
            max_value.value = max_val
            
            # Update filter status
            with filter_status:
                filter_status.clear_output(wait=True)
                display(widgets.HTML(
                    f"<p>Ready to filter <b>{col}</b></p>"
                    f"<p>Data range: {min_val:.4g} to {max_val:.4g}</p>"
                ))
    
    # Function to apply filter
    def apply_filter(b):
        global data
        col = column_selector.value
        
        if col and col in data.columns:
            # Get min and max values directly from the input fields
            min_val = min_value.value
            max_val = max_value.value
            
            # Check if min is smaller than max
            if min_val > max_val:
                with filter_status:
                    filter_status.clear_output(wait=True)
                    display(widgets.HTML(
                        f"<p style='color: red'>Error: Minimum value ({min_val:.4g}) cannot be greater than maximum value ({max_val:.4g})</p>"
                    ))
                return
                
            # Filter the data
            data = data[(data[col] >= min_val) & (data[col] <= max_val)]
            
            # Update filter status
            with filter_status:
                filter_status.clear_output(wait=True)
                display(widgets.HTML(
                    f"<p>Applied filter: <b>{col}</b> between {min_val:.4g} and {max_val:.4g}</p>"
                    f"<p>Remaining rows: {len(data)} out of {len(original_data)}</p>"
                ))
            
            # Update the plot widgets if they exist
            with filter_output:
                filter_output.clear_output(wait=True)
                display(widgets.HTML(f"<p>Data filtered: {len(data)} rows remaining</p>"))
                # Show a small sample of the filtered data
                display(data.head(5))
    
    # Function to reset all filters
    def reset_filters(b):
        global data
        data = original_data.copy()
        
        # Update input values for current column
        if column_selector.value:
            update_input_values(type('obj', (object,), {'new': column_selector.value}))
        
        # Update filter status
        with filter_status:
            filter_status.clear_output(wait=True)
            display(widgets.HTML("<p><b>All filters reset!</b> Original data restored.</p>"))
        
        # Update the filtered data display
        with filter_output:
            filter_output.clear_output(wait=True)
            display(widgets.HTML(f"<p>Data reset to original: {len(data)} rows</p>"))
            display(data.head(5))
    
    # Function to show current filtered data
    def show_filtered_data(b):
        with filter_output:
            filter_output.clear_output(wait=True)
            display(widgets.HTML(f"<h4>Current Filtered Data ({len(data)} rows)</h4>"))
            display(data.head(10))
            if len(data) > 10:
                display(widgets.HTML(f"<p>Showing first 10 of {len(data)} rows</p>"))
            
            # Show filter summary
            if len(data) < len(original_data):
                display(widgets.HTML(
                    f"<p><b>Filters applied:</b> {len(original_data) - len(data)} "
                    f"rows filtered out ({len(data) / len(original_data) * 100:.1f}% of original data remaining)</p>"
                ))
    
    # Connect event handlers
    column_selector.observe(update_input_values, names='value')
    apply_button.on_click(apply_filter)
    reset_button.on_click(reset_filters)
    show_data_button.on_click(show_filtered_data)
    
    # Initialize input fields if a column is selected
    if column_selector.value:
        update_input_values(type('obj', (object,), {'new': column_selector.value}))
    
    # Layout the widgets
    filter_controls = widgets.VBox([
        widgets.HTML("<h3>Filter Data</h3>"),
        widgets.HTML("<p>Select a column and specify the range to filter:</p>"),
        column_selector,
        widgets.HBox([min_value, max_value]),
        widgets.HBox([apply_button, reset_button, show_data_button]),
        filter_status
    ])
    
    return widgets.VBox([filter_controls, filter_output])

def create_plotting_widgets():
    """Create widgets for plotting functionality"""
    global data
    
    if data is None:
        return widgets.HTML("No data available for plotting.")
    
    # Get available columns for plotting
    numeric_cols = data.select_dtypes(include=['float64', 'int64']).columns.tolist()
    category_cols = ['sample_id', 'variation', 'name']
    all_cols = numeric_cols + category_cols
    
    # Create dropdowns for X and Y axes
    x_dropdown = widgets.Dropdown(
        options=all_cols,
        value=numeric_cols[0] if numeric_cols else all_cols[0],
        description='X-axis:',
        tooltip="Select column for X-axis"
    )
    
    y_dropdown = widgets.Dropdown(
        options=numeric_cols,
        value=numeric_cols[1] if len(numeric_cols) > 1 else numeric_cols[0],
        description='Y-axis:',
        tooltip="Select column for Y-axis"
    )
    
    # Create dropdown for color column
    color_dropdown = widgets.Dropdown(
        options=['None'] + category_cols,
        value='variation',
        description='Color by:',
        tooltip="Select column to color points by"
    )
    
    # Create dropdown for plot type
    plot_type_dropdown = widgets.Dropdown(
        options=['Scatter plot', 'Box plot'],
        value='Scatter plot',
        description='Plot type:',
        tooltip="Select type of plot to display"
    )
    
    # Create button to generate the plot
    plot_button = widgets.Button(
        description='Generate Plot',
        button_style='success',
        tooltip='Click to generate the plot'
    )
    
    # Create plot output area
    plot_output = widgets.Output()
    
    # Define the plotting function
    def on_plot_button_clicked(b):
        with plot_output:
            plot_output.clear_output(wait=True)
            x_col = x_dropdown.value
            y_col = y_dropdown.value
            color_col = None if color_dropdown.value == 'None' else color_dropdown.value
            plot_type = plot_type_dropdown.value
            
            if plot_type == 'Scatter plot':
                fig = px.scatter(
                    data, 
                    x=x_col, 
                    y=y_col, 
                    color=color_col,
                    hover_data=['name', 'sample_id'],
                    title=f'Scatter Plot: {y_col} vs {x_col}'
                )
                fig.update_layout(height=600, width=800)
                display(fig)
                
            elif plot_type == 'Box plot':
                if color_col and color_col != 'None':
                    fig = px.box(
                        data, 
                        x=color_col, 
                        y=y_col, 
                        title=f'Box Plot of {y_col} by {color_col}'
                    )
                    fig.update_layout(height=600, width=800)
                    display(fig)
                else:
                    fig = px.box(
                        data, 
                        y=y_col, 
                        title=f'Box Plot of {y_col}'
                    )
                    fig.update_layout(height=600, width=800)
                    display(fig)
    
    # Connect the button to the function
    plot_button.on_click(on_plot_button_clicked)
    
    # Layout the widgets
    controls = widgets.VBox([
        widgets.HBox([x_dropdown, y_dropdown]),
        widgets.HBox([color_dropdown, plot_type_dropdown]),
        plot_button
    ])
    
    return widgets.VBox([controls, plot_output])

def on_load_data_clicked(batch_ids_selector):
    #global dictionary to hold data
    global data, original_data
    dynamic_content.clear_output()
    with out:
        out.clear_output()
        print("Loading Data")

        try_sample_ids = get_ids_in_batch(url, token, batch_ids_selector.value)

        #extract EQE here
        identifiers = get_sample_description(url, token, list(try_sample_ids))
        data = get_abspl_data(try_sample_ids, identifiers)

        # Check if EQE data was found
        if data is None:
            out.clear_output()
            print("The batches selected don't contain any AbsPL measurements")
            return

        # Store original data for filter reset functionality
        original_data = data.copy()
        
        out.clear_output()
        print("Data Loaded")
        
        # Create and display plotting widgets once data is loaded
        with dynamic_content:
            dynamic_content.clear_output(wait=True)
            
            # Display data summary
            display(widgets.HTML("<h3>Data Summary</h3>"))
            display(data.describe())
            
            # Display filtering options immediately after data loading
            display(widgets.HTML("<h3>Data Filtering</h3>"))
            filtering_widgets = create_data_filters()
            display(filtering_widgets)
            
            # Display plotting widgets
            display(widgets.HTML("<h3>Plotting Tools</h3>"))
            plotting_widgets = create_plotting_widgets()
            display(plotting_widgets)

# BATCH SELECTION WITH OPTIONAL FILTERING
def create_batch_selection_with_optional_filtering():
    """
    Create batch selection widget with optional filtering button
    """
    # Create the original batch selection widget (fast)
    original_batch_widget = batch_selection.create_batch_selection(url, token, on_load_data_clicked)
    
    # Get the batch selector from the original widget to count total batches
    batch_selector = None
    for child in original_batch_widget.children:
        if isinstance(child, widgets.SelectMultiple):
            batch_selector = child
            break
    
    total_batches = len(batch_selector.options) if batch_selector else 0
    
    # Create filter button
    filter_button = widgets.Button(
        description=f"🔍 Filter to show only batches with AbsPL data",
        button_style='info',
        tooltip=f'Click to filter {total_batches} batches (this may take a few minutes)',
        layout=widgets.Layout(width='400px')
    )
    
    # Create status output
    filter_status = widgets.Output()
    
    # Filter function
    def start_filtering(b):
        filter_button.disabled = True
        filter_button.description = "🔄 Filtering in progress..."
        
        with filter_status:
            filter_status.clear_output(wait=True)
            print("Finding batches with AbsPL data...")
            
            # Get all batch IDs using the same filtering as the original batch_selection
            batch_ids_list_tmp = list(get_batch_ids(url, token))
            all_batch_ids = []
            for batch in batch_ids_list_tmp:
                if "_".join(batch.split("_")[:-1]) in batch_ids_list_tmp:
                    continue
                all_batch_ids.append(batch)
            
            print(f"Testing {len(all_batch_ids)} batches...")
            
            valid_batches = []
            
            for i, batch_id in enumerate(all_batch_ids):
                # Update progress every 10 batches
                if i % 10 == 0 or i == len(all_batch_ids) - 1:
                    filter_status.clear_output(wait=True)
                    print(f"Progress: {i+1}/{len(all_batch_ids)} - Found {len(valid_batches)} valid batches")
                    print(f"Currently testing: {batch_id}")
                
                try:
                    # Get sample IDs for this single batch
                    sample_ids = get_ids_in_batch(url, token, [batch_id])
                    
                    if sample_ids:
                        # Get sample descriptions
                        identifiers = get_sample_description(url, token, list(sample_ids))
                        
                        # Test get_abspl_data - if it returns data (not None), keep this batch
                        abspl_data = get_abspl_data(sample_ids, identifiers)
                        
                        if abspl_data is not None:
                            valid_batches.append(batch_id)
                            filter_status.clear_output(wait=True)
                            print(f"✅ Found valid batch: {batch_id} ({len(abspl_data)} rows)")
                            print(f"Total found so far: {len(valid_batches)}")
                except:
                    # Skip batches that cause errors
                    continue
            
            # Update the original widget's options
            if batch_selector:
                batch_selector.options = valid_batches
            
            # Show final results
            filter_status.clear_output(wait=True)
            print("="*60)
            print("FILTERING COMPLETE")
            print("="*60)
            print(f"✅ Found {len(valid_batches)} batches with AbsPL data out of {total_batches} total")
            if len(valid_batches) > 0:
                print(f"Valid batches: {valid_batches}")
            else:
                print("⚠️  No batches with AbsPL data found!")
            
            # Update button
            filter_button.description = f"✅ Filtering complete - {len(valid_batches)} valid batches found"
            filter_button.disabled = True
            
            # Add info to the widget
            info_html = widgets.HTML(
                value=f"<p><b>Showing {len(valid_batches)} of {total_batches} batches with confirmed AbsPL data</b></p>"
            )
            original_batch_widget.children = (info_html,) + original_batch_widget.children
    
    # Connect the button
    filter_button.on_click(start_filtering)
    
    # Create the complete widget
    complete_widget = widgets.VBox([
        widgets.HTML(f"<p>Select batches from all {total_batches} available batches, or use the filter button below:</p>"),
        filter_button,
        filter_status,
        original_batch_widget
    ])
    
    return complete_widget

# MAIN EXECUTION
display(plotting_utils.create_manual("eqe_manual.md"))

# Create and display the batch selection widget with optional filtering
batch_widget = create_batch_selection_with_optional_filtering()
display(batch_widget)

display(out)
display(dynamic_content)  # This will be updated dynamically with the variables menu

Output()

Output()

In [ ]:
# Additional functions for data visualization and export

def create_data_table_view():
    """Create a data table view with filtering capabilities"""
    global data
    
    if data is None:
        return widgets.HTML("No data available for displaying.")
    
    # Create output for the table display
    table_output = widgets.Output()
    
    # Create column selection for table view
    columns = data.columns.tolist()
    column_selector = widgets.SelectMultiple(
        options=columns,
        value=columns[:5],  # Default to first 5 columns
        description='Columns:',
        disabled=False,
        layout=widgets.Layout(width='50%', height='100px')
    )
    
    # Add button to update table view
    update_button = widgets.Button(
        description='Update Table',
        button_style='info',
        tooltip='Click to update the table view'
    )
    
    # Add export button
    export_button = widgets.Button(
        description='Export to CSV',
        button_style='warning',
        tooltip='Click to export current data to CSV'
    )
    
    # Function to update table view
    def update_table_view(b):
        with table_output:
            table_output.clear_output(wait=True)
            if column_selector.value:
                display(data[list(column_selector.value)].head(20))
                display(widgets.HTML(f"<p>Showing top 20 rows of {len(data)} total rows</p>"))
            else:
                display(widgets.HTML("<p>Please select at least one column to display</p>"))
    
    # Function to export data to CSV
    def trigger_download(text, filename, kind='text/json'):
        content_b64 = base64.b64encode(text.encode()).decode()
        data_url = f'data:{kind};charset=utf-8;base64,{content_b64}'
        js_code = f"""
            var a = document.createElement('a');
            a.setAttribute('download', '{filename}');
            a.setAttribute('href', '{data_url}');
            a.click()
        """
        with download_content:
            download_content.clear_output()
            display(HTML(f'<script>{js_code}</script>'))

    def export_to_csv(e=None):
        abspl_data = io.StringIO()
        data.to_csv(abspl_data)
        timestamp = pd.Timestamp.now().strftime("%Y%m%d_%H%M%S")
        filename = f"abspl_data_export_{timestamp}.csv"
        trigger_download(abspl_data.getvalue(), filename, kind='text/plain')
        with table_output:
            table_output.clear_output(wait=True)
            display(widgets.HTML(f"<p>Data exported to {filename}</p>"))
            update_table_view(None)
    
    # Connect buttons to functions
    update_button.on_click(update_table_view)
    export_button.on_click(export_to_csv)
    
    # Layout the widgets
    controls = widgets.VBox([
        widgets.HTML("<h4>Select columns to display:</h4>"),
        column_selector,
        widgets.HBox([update_button, export_button])
    ])
    
    # Initialize table view
    with table_output:
        if column_selector.value:
            display(data[list(column_selector.value)].head(20))
            display(widgets.HTML(f"<p>Showing top 20 rows of {len(data)} total rows</p>"))
    
    return widgets.VBox([controls, table_output])

def create_statistics_view():
    """Create a view for statistical analysis of the data"""
    global data
    
    if data is None:
        return widgets.HTML("No data available for statistics.")
    
    # Create output for statistics display
    stats_output = widgets.Output()
    
    # Get numeric columns for statistics
    numeric_cols = data.select_dtypes(include=['float64', 'int64']).columns.tolist()
    
    # Create column selection for statistics
    column_selector = widgets.Dropdown(
        options=numeric_cols,
        value=numeric_cols[0] if numeric_cols else None,
        description='Column:',
        disabled=False
    )
    
    # Create groupby column selection
    category_cols = ['sample_id', 'variation', 'name']
    groupby_selector = widgets.Dropdown(
        options=['None'] + category_cols,
        value='variation',
        description='Group by:',
        disabled=False
    )
    
    # Add button to calculate statistics
    stats_button = widgets.Button(
        description='Calculate Stats',
        button_style='info',
        tooltip='Click to calculate statistics'
    )
    
    # Function to calculate and display statistics
    def calculate_statistics(b):
        with stats_output:
            stats_output.clear_output(wait=True)
            col = column_selector.value
            groupby = groupby_selector.value
            
            if col:
                # Overall statistics
                display(widgets.HTML(f"<h4>Overall Statistics for {col}</h4>"))
                stats = data[col].describe()
                display(stats)
                
                # Grouped statistics if groupby is selected
                if groupby != 'None':
                    display(widgets.HTML(f"<h4>Statistics for {col} grouped by {groupby}</h4>"))
                    grouped_stats = data.groupby(groupby)[col].describe()
                    display(grouped_stats)
                    
                    # Create a comparison boxplot
                    fig = px.box(data, x=groupby, y=col, title=f"Comparison of {col} by {groupby}")
                    display(fig)
            else:
                display(widgets.HTML("<p>Please select a column for statistics</p>"))
    
    # Connect button to function
    stats_button.on_click(calculate_statistics)
    
    # Layout the widgets
    controls = widgets.VBox([
        widgets.HBox([column_selector, groupby_selector]),
        stats_button
    ])
    
    return widgets.VBox([controls, stats_output])

# Create tabs for different functionalities
def display_advanced_features():
    global data
    
    if data is None:
        return widgets.HTML("<h3>Please load data first</h3>")
    
    # Create tabs
    tab1 = create_data_table_view()
    tab2 = create_statistics_view()
    
    tabs = widgets.Tab()
    tabs.children = [tab1, tab2]
    tabs.titles = ['Data Table', 'Statistics']
    tabs.set_title(0, 'Data Table')
    tabs.set_title(1, 'Statistics')
    
    return tabs

# Button to show/hide advanced features
advanced_button = widgets.Button(
    description='Toggle Advanced Features',
    button_style='primary',
    tooltip='Click to show/hide advanced data features'
)

advanced_output = widgets.Output()

def on_advanced_button_clicked(b):
    with advanced_output:
        advanced_output.clear_output(wait=True)
        display(display_advanced_features())

advanced_button.on_click(on_advanced_button_clicked)

display(widgets.HTML("<h2>Advanced Data Analysis</h2>"))
display(advanced_button)
display(advanced_output)
download_content = widgets.Output()
download_area = widgets.VBox([download_content])
display(download_area)

HTML(value='<h2>Advanced Data Analysis</h2>')

Button(button_style='primary', description='Toggle Advanced Features', style=ButtonStyle(), tooltip='Click to …

Output()

In [ ]:
# Check data structure for wavelength and luminescence_flux_density columns
if data is not None:
    print("Data types of columns:")
    print(data.dtypes)
    
    print("\nSample from wavelength column:")
    if 'wavelength' in data.columns:
        sample_wavelength = data['wavelength'].iloc[0]
        print(f"Type: {type(sample_wavelength)}")
        print(f"Length: {len(sample_wavelength) if hasattr(sample_wavelength, '__len__') else 'Not a list'}")
        print(f"Sample values: {sample_wavelength[:5] if hasattr(sample_wavelength, '__getitem__') else sample_wavelength}")
    
    print("\nSample from luminescence_flux_density column:")
    if 'luminescence_flux_density' in data.columns:
        sample_flux = data['luminescence_flux_density'].iloc[0]
        print(f"Type: {type(sample_flux)}")
        print(f"Length: {len(sample_flux) if hasattr(sample_flux, '__len__') else 'Not a list'}")
        print(f"Sample values: {sample_flux[:5] if hasattr(sample_flux, '__getitem__') else sample_flux}")
else:
    print("No data available. Please load data first.")

In [ ]:
# Create specialized plotting function for wavelength vs. luminescence_flux_density
def create_spectral_plot():
    """Create a specialized plot for wavelength vs. luminescence_flux_density data"""
    global data
    
    if data is None:
        return widgets.HTML("<h3>Please load data first</h3>")
    
    # Check if required columns exist
    if 'wavelength' not in data.columns or 'luminescence_flux_density' not in data.columns:
        return widgets.HTML("<h3>Required columns 'wavelength' and 'luminescence_flux_density' not found</h3>")
    
    # Output area for plots
    plot_output = widgets.Output()
    plot_status = widgets.Output()
    
    # Get unique variations for color selection
    variations = data['variation'].unique().tolist()
    
    # Create selection for variations to plot
    variation_selector = widgets.SelectMultiple(
        options=variations,
        value=variations[:min(5, len(variations))],  # Default to first 5 variations or fewer
        description='Variations:',
        disabled=False,
        layout=widgets.Layout(width='60%', height='120px')
    )
    
    # Add option for plot scale
    scale_selector = widgets.RadioButtons(
        options=['linear', 'log'],
        value='linear',
        description='Y-axis scale:',
        disabled=False
    )
    
    # Add option to normalize the data
    normalize_checkbox = widgets.Checkbox(
        value=False,
        description='Normalize spectra',
        disabled=False,
        indent=False
    )
    
    # Create button to generate the plot
    plot_button = widgets.Button(
        description='Generate Spectral Plot',
        button_style='success',
        tooltip='Click to generate the wavelength vs. luminescence_flux_density plot'
    )
    
    # Define plotting function
    def on_plot_button_clicked(b):
        with plot_output:
            plot_output.clear_output(wait=True)
            
            # Get selected variations
            selected_variations = variation_selector.value
            if not selected_variations:
                display(widgets.HTML("<p>Please select at least one variation to plot</p>"))
                return
            
            try:
                with plot_status:
                    plot_status.clear_output(wait=True)
                    display(widgets.HTML("<p>Generating plot...</p>"))
                
                # Create a figure with plotly
                fig = go.Figure()
                
                # Generate a color map for variations to ensure consistent colors
                # We'll create a color scale with evenly spaced colors
                import plotly.colors as pc
                colorscale = px.colors.qualitative.Plotly  # Using Plotly's qualitative color scale
                variation_colors = {var: colorscale[i % len(colorscale)] for i, var in enumerate(selected_variations)}
                
                # Group the data by variation to process all samples of the same variation together
                filtered_data = data[data['variation'].isin(selected_variations)]
                
                # Process each variation separately to ensure consistent color
                for variation in selected_variations:
                    variation_data = filtered_data[filtered_data['variation'] == variation]
                    color = variation_colors[variation]
                    
                    # Process each sample in this variation
                    for i, row in variation_data.iterrows():
                        # Extract wavelength and luminescence data
                        try:
                            wavelengths = row['wavelength']
                            luminescence = row['luminescence_flux_density']
                            
                            # Normalize if requested
                            if normalize_checkbox.value and max(luminescence) > 0:
                                luminescence = [l / max(luminescence) for l in luminescence]
                            
                            # Add trace for this sample with consistent color for the same variation
                            name = f"{variation} - {row['name']}" if row['name'] else f"{variation} {i}"
                            fig.add_trace(go.Scatter(
                                x=wavelengths, 
                                y=luminescence,
                                mode='lines',
                                name=name,
                                line=dict(width=2, color=color),
                                hovertemplate='Wavelength: %{x:.2f} nm<br>Luminescence: %{y:.4e}<extra>%{fullData.name}</extra>'
                            ))
                        except (TypeError, ValueError, IndexError) as e:
                            display(widgets.HTML(f"<p style='color:red'>Error processing row {i}: {str(e)}</p>"))
                            continue
                
                # Update layout
                y_title = 'Normalized Luminescence' if normalize_checkbox.value else 'Luminescence Flux Density'
                
                fig.update_layout(
                    title='Wavelength vs. Luminescence Flux Density',
                    xaxis_title='Wavelength (nm)',
                    yaxis_title=y_title,
                    yaxis_type=scale_selector.value,
                    height=600,
                    width=900,
                    template='plotly_white',
                    legend_title_text='Sample',
                    hovermode='closest'
                )
                
                # Add x-axis range slider
                fig.update_layout(
                    xaxis=dict(
                        rangeslider=dict(visible=True),
                        type='linear'
                    )
                )
                
                display(fig)
                
                with plot_status:
                    plot_status.clear_output(wait=True)
                    display(widgets.HTML("<p>Plot generated successfully</p>"))
                    
            except Exception as e:
                with plot_status:
                    plot_status.clear_output(wait=True)
                    display(widgets.HTML(f"<p style='color:red'>Error generating plot: {str(e)}</p>"))
    
    # Connect the button to the function
    plot_button.on_click(on_plot_button_clicked)
    
    # Layout the widgets
    controls = widgets.VBox([
        widgets.HTML("<h3>Spectral Plot</h3>"),
        widgets.HTML("<p>Select variations to plot:</p>"),
        variation_selector,
        widgets.HBox([scale_selector, normalize_checkbox]),
        plot_button,
        plot_status
    ])
    
    return widgets.VBox([controls, plot_output])

# Create a button to show/hide the spectral plot functionality
spectral_button = widgets.Button(
    description='Spectral Plot',
    button_style='info',
    tooltip='Click to show/hide spectral plot tools'
)

spectral_output = widgets.Output()

def on_spectral_button_clicked(b):
    with spectral_output:
        spectral_output.clear_output(wait=True)
        display(create_spectral_plot())

spectral_button.on_click(on_spectral_button_clicked)

display(widgets.HTML("<h2>Wavelength vs. Luminescence Flux Density Plot</h2>"))
display(widgets.HTML("<p>Click below to visualize the spectral data with interactive plotting tools.</p>"))
display(spectral_button)
display(spectral_output)